Decorators 101

In [9]:
#Example

def deco(func):
    def inner():
        print('running inner')
        func()
    return inner

@deco
def target():
    print(f'running target')

In [10]:
target()

running inner
running target


In [11]:
target

<function __main__.deco.<locals>.inner()>

In [12]:
import registration  #decorators usually run at runtime
#refer to registration.py

running register<function f1 at 0x7f506e511a80>
running register<function f2 at 0x7f506e511c60>
running register<function f3 at 0x7f506e512ac0>


In [6]:
b = 6
def f2(a):
    print(a);
    print(b);
    b=9;

In [8]:
#f2(3) runs to error because the b here refers to nothing while it is quite tempting to confuse the inner b with the b =6, it can only access and modify that variable if we refer to that variable explicitly

In [24]:
#The correct response will be
b = 9;
def f3(a):
    global b;
    c=0;
    c+=b
    print(a)
    c=7
    print(c)
    return b


In [ ]:
f3(5)

#The module global scope
# It is made of names assigned to values outside of any class or function block.

#The f3 function local scope
#Made of names assigned to values as parameters, or directly in the body of the function.

#using the dis module we will dissasemble the bytecode of python functions
from dis import dis
dis(f2)

In [ ]:
dis(f3)

In [37]:
class Averager():
    def __init__(self):
        self.series = []

    def __call__(self, *args: any, **kwds: any) -> any:
        self.series.append(*args)
        total = sum(self.series)
        return total/len(self.series)

In [40]:
avg = Averager()
avg(10)
avg(11)
avg(99)

40.0

In [1]:
#This is a functional implementation using the higher order function

def make_averager():
    series = []

    def averager(new_value:any) -> any:
        series.append(int(new_value))
        total = sum(series)
        return total/len(series)
    return averager

In [2]:
I = make_averager()
I('1')
I(9)

5.0

In [7]:
I.__code__.co_varnames
#way to check the covariables names

('new_value', 'total')

In [9]:
I.__code__.co_freevars

('series',)

In [15]:
I.__closure__[0].cell_contents
#shows the cell contents

[1, 9]

The nonlocal Declaration

In [28]:
def make_averager():
    count = 0
    total = 0

    def averager(new_value):
        nonlocal count,total
        count += 1
        total += new_value
        return total / count

    return averager


#if the nonlocal term was never added we couldnt modify the count as well as the total cos they are immutable types, hence in order to make theme behave as  local we can add the nonlocal term to it

In [31]:
from dis import dis
dis(make_averager())

              0 COPY_FREE_VARS           2

  6           2 RESUME                   0

  8           4 LOAD_DEREF               2 (count)
              6 LOAD_CONST               1 (1)
              8 BINARY_OP               13 (+=)
             12 STORE_DEREF              2 (count)

  9          14 LOAD_DEREF               3 (total)
             16 LOAD_FAST                0 (new_value)
             18 BINARY_OP               13 (+=)
             22 STORE_DEREF              3 (total)

 10          24 LOAD_FAST                1 (pund)
             26 LOAD_CONST               1 (1)
             28 BINARY_OP               13 (+=)
             32 STORE_FAST               1 (pund)

 11          34 LOAD_DEREF               3 (total)
             36 LOAD_DEREF               2 (count)
             38 BINARY_OP               11 (/)
             42 RETURN_VALUE


Implementing a Simple Decorator

In [33]:
from clockdeco0 import clock
import time

@clock
def snooze(seconds):
    time.sleep(seconds)


@clock
def factorial(arg:int) -> int:
    return 1 if arg<2 else arg*factorial(arg-1)

In [35]:
snooze(5)

[5.00011845s] snooze (5) -> None


In [36]:
print('*'*40, 'calling snooze')
snooze(5)
print('*'*40,'calling factorial(6)')
factorial(6)

**************************************** calling snooze
[5.00012432s] snooze (5) -> None
**************************************** calling factorial(6)
[0.00000220s] factorial (1) -> 1
[0.00007474s] factorial (2) -> 2
[0.00014452s] factorial (3) -> 6
[0.00019044s] factorial (4) -> 24
[0.00024251s] factorial (5) -> 120
[0.00029989s] factorial (6) -> 720


720

Decorators in the Standard Library

In [1]:
#Memoization with functools.cache

#It is an optimization technique that works by savign the results of previous invocations of an expensive function, avoiding repeat computations on previously used arguments.

In [2]:
#A good demonstration is to apply@cache to the painfullt slow recursive function to generate the nth number in the Fibonacci sequence

In [3]:
from clockdeco import clock

@clock
def fibonacci(n):
    if n<2:
        return n
    return fibonacci(n-2) + fibonacci(n-1)

In [6]:
fibonacci(6)

[0.00000065s] fibonacci(0) -> 0
[0.00000073s] fibonacci(1) -> 1
[0.00008239s] fibonacci(2) -> 1
[0.00000060s] fibonacci(1) -> 1
[0.00000061s] fibonacci(0) -> 0
[0.00000066s] fibonacci(1) -> 1
[0.00002957s] fibonacci(2) -> 1
[0.00005788s] fibonacci(3) -> 2
[0.00016855s] fibonacci(4) -> 3
[0.00000059s] fibonacci(1) -> 1
[0.00000060s] fibonacci(0) -> 0
[0.00000062s] fibonacci(1) -> 1
[0.00002761s] fibonacci(2) -> 1
[0.00005487s] fibonacci(3) -> 2
[0.00000059s] fibonacci(0) -> 0
[0.00000062s] fibonacci(1) -> 1
[0.00002753s] fibonacci(2) -> 1
[0.00000057s] fibonacci(1) -> 1
[0.00000064s] fibonacci(0) -> 0
[0.00000064s] fibonacci(1) -> 1
[0.00002781s] fibonacci(2) -> 1
[0.00005511s] fibonacci(3) -> 2
[0.00010938s] fibonacci(4) -> 3
[0.00019101s] fibonacci(5) -> 5
[0.00038666s] fibonacci(6) -> 8


8

In [7]:
#but by adjusting jsut the two line to use cache performance in improved

In [8]:
import functools
from clockdeco import clock

@functools.cache
@clock
def fibonacci(n):
    if n<2:
        return n
    return fibonacci(n-2) + fibonacci(n-1)

In [9]:
fibonacci(6)

[0.00000032s] fibonacci(0) -> 0
[0.00000070s] fibonacci(1) -> 1
[0.00006893s] fibonacci(2) -> 1
[0.00000097s] fibonacci(3) -> 2
[0.00009026s] fibonacci(4) -> 3
[0.00000056s] fibonacci(5) -> 5
[0.00010821s] fibonacci(6) -> 8


8

Single Dispatch Generic Functions

In [10]:
#singledispatch creates a custom function to bundle several functions into a generic functiopn

In [11]:
from functools import singledispatch
from collections import abc
import fractions
import decimal
import html
import numbers

@singledispatch
def htmlize(obj: object) -> str:
    content = html.escape(repr(obj))
    return f'<pre>{content}</pre>'


@htmlize.register
def _(text: str) -> str:
    content = html.escape(text).replace('\n', '<br/>\n')
    return f'<p>{content}</p>'

@htmlize.register
def _(seq: abc.Sequence) -> str:
    inner = '</li>\n<li>'.join(htmlize(item) for item in seq)
    return '<ul>\n<li>' + inner + '</li>\n</ul>'

@htmlize.register
def _(n: numbers.Integral) -> str:
    return f'<pre>{n} (0x{n:x})</pre>'

@htmlize.register  
def _(n: bool) -> str:
    return f'<pre>{n}</pre>'

@htmlize.register(fractions.Fraction)  
def _(x) -> str:
    frac = fractions.Fraction(x)
    return f'<pre>{frac.numerator}/{frac.denominator}</pre>'

@htmlize.register(decimal.Decimal)  
@htmlize.register(float)
def _(x) -> str:
    frac = fractions.Fraction(x).limit_denominator()
    return f'<pre>{x} ({frac.numerator}/{frac.denominator})</pre>'

In [12]:
htmlize({1,2,3})

'<pre>{1, 2, 3}</pre>'

In [13]:
htmlize('abs')

'<p>abs</p>'

In [14]:
htmlize(abs)

'<pre>&lt;built-in function abs&gt;</pre>'

In [15]:
print(htmlize(['alpha',66,{3,2,1}]))

<ul>
<li><p>alpha</p></li>
<li><pre>66 (0x42)</pre></li>
<li><pre>{1, 2, 3}</pre></li>
</ul>


Parameterized Decorators

In [17]:
registry = set()

def register(active=True):
    def decorate(func):
        print('running register'
            f'(active={active})->decorate({func})')
        if active:
            registry.add(func)
        else:
            registry.discard(func)

        return func
    return decorate

@register(active=False)
def f1():
    print('running f2()')

@register()
def f2():
    print('running f2()')
def f3():
    print('running f3()')

running register(active=False)->decorate(<function f1 at 0x7f8de7703e20>)
running register(active=True)->decorate(<function f2 at 0x7f8de7702160>)


In [20]:
from Module_clockdeco_cls import clock

@clock
def snooze(seconds):
    time.sleep(seconds)



In [21]:
for i in range(4):
    snooze(3)

In [16]:
class function_wrapper():
    def __init__(self, wrapped):
        self.wrapped = wrapped
    def __call__(self, *args, **kwargs):
        return self.wrapped(*args, **kwargs)

@function_wrapper
def function():
    return f'This is the function'

In [19]:
function_wrapper(function)()

'This is the function'